<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- Calculation of Present Values
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- [Creation of Ifrs Variables](../Import/5ImportScope-ToIfrsVar)
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the calculation of present values.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "1ImportScope-Identities"

# Discounting calculation

The calculation of IFRS 17 figures is based on cumulated discounted cash flows.

## Yield Curves

The Yield Curves used for the discounting calculations are functions of the [Currency](../DataModel/DataStructure#currency), the [Data Node](../DataModel/DataStructure#data-node) and the [Economic Basis](../DataModel/DataStructure#economic-basis).

In particular:
- For the **Locked-in** economic basis, the yield curve used is the latest available as per end of the DataNode's inception year;
- Whereas for the **Current** economic base, the yield curve used is the latest available as per the current period.

The algorithm which retrieves the latest available yield curve is [here](../Utils/Queries#yield-curve).

## Interest and Discount Rates and Factors

Calculation of present values can be performed using Current or Locked-in rates (locked at the inception year of the data node). During calculation, this is controlled by the Economic Basis parameter which is either set to C for Current rates or L for Locked-in rates. In addition, the Economic Basis N identifies absence of discounting, leading to computation of Nominal values.

<br> The Monthly Rate scope returns both the Discounting and Interest factors used in the calculation of the corresponding quantities. When Economic basis is N empty arrays are returned. For the other values of Economic Basis, the factors used for discounting have the same granularity as the cash flow, i.e. monthly. The yield curves have yearly granularity, so the annual Interest factor is 1 + interest rate. The monthly Interest Interest and Discount factors are obtained from the annual factors such that the product of 12 months results in the annual factors, as follows:

$$
\text{Discount}_i = ( 1 + \text{YC}_i ) ^{-\frac{1}{12}} ~,
$$
and 
$$
\text{Interest}_i = ( 1 + \text{YC}_i) ^{\frac{1}{12}} ~,
$$

where the index $i$ denotes years.

In [0]:
public interface MonthlyRate : IScope<ImportIdentity, ImportStorage>
{
    private string EconomicBasis => GetContext();
    
    private double[] YearlyYieldCurve => EconomicBasis switch {
        EconomicBases.N => new [] { 0d },
        _ => GetStorage().GetYearlyYieldCurve(Identity, EconomicBasis),
    };
    
    double[] Interest => YearlyYieldCurve.Select(rate => Math.Pow(1d + rate, 1d / 12d)).ToArray();   
                        
    double[] Discount => Interest.Select(x => Math.Pow(x, -1)).ToArray();
}

# Nominal Values

The nominal cash flow values correspond to the cash flows provided in the cash flow input file. 
<br> These values are stored in the database as [RawVariable](../DataModel/DataStructure#raw-variables).
<br> Refer to the ReferenceAocStep of the AocStructure calculation to identify the correct AoC Type and Novelty to retrieve.
<br> Due to the Credit Default Risk of a reinsurance partner, the logic to compute the Nominal Cash flows for this Amount Type must be defined separately.

In [0]:
public interface NominalCashflow : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<NominalCashflow>(s => s
            .WithApplicability<EmptyNominalCashflow>(x =>
                (x.Identity.Id.AocType != AocTypes.CL && x.Identity.Id.AocType != AocTypes.EOP) && // if AocType is NOT CL AND NOT EOP AND
                x.Identity.Id.Novelty != Novelties.I && // if Novelty is NOT inforce AND
                x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod) >= MonthInAYear && // if it is projection >= 1 Year AND
                !(x.Identity.AccidentYear.HasValue && MonthInAYear * x.Identity.AccidentYear >= (MonthInAYear * x.GetStorage().CurrentReportingPeriod.Year + x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod))) // if it DOES NOT (have AY and with AY >= than projected FY)
            )
            .WithApplicability<EmptyNominalCashflow>(x =>
                (x.Identity.Id.AocType == AocTypes.BOP || x.Identity.Id.AocType == AocTypes.CF || x.Identity.Id.AocType == AocTypes.IA) && // if AocType is BOP, CF or IA (or not in telescopic) AND
                x.Identity.Id.Novelty == Novelties.I && // if Novelty is inforce AND
                x.Identity.Id.LiabilityType == LiabilityTypes.LIC && // if LiabilityType is LIC AND
                x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod) >= MonthInAYear &&  // if it is projection >= 1 Year AND
                (x.Identity.AccidentYear.HasValue && MonthInAYear * x.Identity.AccidentYear >= (MonthInAYear * x.GetStorage().CurrentReportingPeriod.Year + x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod))) // if it DOES (have AY and with AY >= than projected FY)
            )
            .WithApplicability<EmptyNominalCashflow>(x =>
                x.Identity.Id.LiabilityType == LiabilityTypes.LRC && // if LiabilityType is LRC
                x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod) >= MonthInAYear && // if it is projection >= 1 Year AND
                (x.Identity.AccidentYear.HasValue && MonthInAYear * x.Identity.AccidentYear < (MonthInAYear * x.GetStorage().CurrentReportingPeriod.Year + x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod))) // if it DOES (have AY and with AY < than projected FY)
            )
            .WithApplicability<EmptyNominalCashflow>(x =>
                (x.Identity.Id.AocType == AocTypes.BOP || x.Identity.Id.AocType == AocTypes.CF || x.Identity.Id.AocType == AocTypes.IA) && // if AocType is BOP, CF or IA (or not in telescopic) AND
                (x.Identity.Id.Novelty != Novelties.I && x.Identity.Id.Novelty != Novelties.C) && // if Novelty is NOT inforce AND
                x.Identity.Id.LiabilityType == LiabilityTypes.LRC && // if LiabilityType is LRC AND
                x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod) >= MonthInAYear && // if it is projection >= 1 Year AND
                (x.Identity.AccidentYear.HasValue && MonthInAYear * x.Identity.AccidentYear >= (MonthInAYear * x.GetStorage().CurrentReportingPeriod.Year + x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod))) // if it DOES (have AY and with AY >= than projected FY)
            )
            .WithApplicability<EmptyNominalCashflow>(x =>
                (x.Identity.Id.AocType == AocTypes.CF) && // if AocType is CF AND
                x.Identity.Id.LiabilityType == LiabilityTypes.LRC && x.Identity.AccidentYear.HasValue && // if LiabilityType is LRC with AY defined
                x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod) == 0 && // if it is projection == 0 AND
                !(MonthInAYear * x.Identity.AccidentYear == (MonthInAYear * x.GetStorage().CurrentReportingPeriod.Year + x.GetStorage().GetShift(x.Identity.Id.ProjectionPeriod))) // if AY == projected FY
            )
            .WithApplicability<CreditDefaultRiskNominalCashflow>(x => x.GetStorage().GetCdr().Contains(x.Identity.AmountType) && x.Identity.Id.AocType == AocTypes.CF)
            .WithApplicability<AllClaimsCashflow>(x => x.GetStorage().GetCdr().Contains(x.Identity.AmountType))
        );

    IEnumerable<AocStep> referenceAocSteps => GetScope<ReferenceAocStep>(Identity.Id).Values;
    double[] Values => referenceAocSteps.Select(refAocStep => GetStorage().GetValues(Identity.Id with {AocType = refAocStep.AocType, Novelty = refAocStep.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear))
                        .AggregateDoubleArray();
}

public interface EmptyNominalCashflow : NominalCashflow
{
    double[] NominalCashflow.Values => Enumerable.Empty<double>().ToArray();
}

public interface CreditDefaultRiskNominalCashflow : NominalCashflow
{
    private double[] NominalClaimsCashflow => referenceAocSteps.SelectMany(refAocStep =>
                            GetStorage().GetClaims()
                            .Select(claim => GetStorage().GetValues(Identity.Id with {AocType = refAocStep.AocType, Novelty = refAocStep.Novelty}, claim, Identity.EstimateType, Identity.AccidentYear)))
                            .AggregateDoubleArray();
                            
    private string cdrBasis => Identity.AmountType == AmountTypes.CDR ? EconomicBases.C : EconomicBases.L;
    private double nonPerformanceRiskRate => GetStorage().GetNonPerformanceRiskRate(Identity.Id, cdrBasis);
                            
    private double[] PvCdrDecumulated { get {
        var ret = new double[NominalClaimsCashflow.Length];
        for (var i = NominalClaimsCashflow.Length - 1; i >= 0; i--)
            ret[i] = Math.Exp(-nonPerformanceRiskRate) * ret.ElementAtOrDefault(i + 1) + NominalClaimsCashflow[i] - NominalClaimsCashflow.ElementAtOrDefault(i + 1);
        return ret; } } 
        
    double[] NominalCashflow.Values => Subtract(PvCdrDecumulated, NominalClaimsCashflow);
}

public interface AllClaimsCashflow : NominalCashflow
{
    double[] NominalCashflow.Values => referenceAocSteps.SelectMany(refAocStep =>
                                        GetStorage().GetClaims()
                                        .Select(claim => GetStorage().GetValues(Identity.Id with {AocType = refAocStep.AocType, Novelty = refAocStep.Novelty}, claim, Identity.EstimateType, Identity.AccidentYear)))
                                        .AggregateDoubleArray();
}

For a given month $i$ they are denoted as $\rm{Nominal}_i$.

# Present Values

Present Values are calculated during the import of the cash flows and stored on the database. They are computed for the relevant Economic Basis, depending on the Valuation Basis.

Their calculation is described in the following sections and is summarized in the $\rm{PV}$ formula [below](#present-value).

## Cumulated Discounted Cash flows

Cumulated and Discounted cash flows $\rm{CDC}$ are computed using the monthly discount rates and in a recursive manner, as follows:

$$
\text{CDC}_i(\text{AoC step}) = \left\{
\begin{array}{cl}
\text{Nominal}_i + \text{CDC}_{i+1} \cdot {\text{Valid Discount}_{\frac{i}{12}}} ~, & \text{if Amount Type's Period Type is Beginning Of Period} \\
\big( \text{Nominal}_i + \text{CDC}_{i+1} \big) \cdot {\text{Valid Discount}_{\frac{i}{12}}} ~, & \text{if Amount Type's Period Type is End Of Period}
\end{array}
\right.
$$

where Transaction Period depends on which Best Estimate value is being computed, in particular on what its [Amount Type](../DataModel/DataStructure#amount-type) is (each Amount Type has its own [Period Type](../Constants/Enums)); and $\text{Valid Discount}$ stands for the fact that in case the Discount Curves are shorter than the required index, then their last element is returned. We also need to flip the sign of the discounted and cumulated values, to create a reserve view and be consistent with the usual [Cash flow Sign Convention](https://en.wikipedia.org/wiki/Cash_flow_sign_convention). 

<br> In case the calculation is triggered for Economic Basis N, the discount factor is assumed to be 1 reducing the above formula to a simple cumulation of the nominal values: 
Cumulated cash flows CC are computed from the Nominal cash flow as follows: 
$$
\text{CC}_i = \sum_{t=i} \text{Nominal}_t 
$$

<br> Also here, the Credit Default Risk contribution is calculated separately. Since it is based on Claims, the Period Type is implicitly defined.
The risk of default is included in the discounting formula as follows. First, we notice that one can explicitly write the discounting recursive relation above as:
$$
\begin{array}{l}
\text{CDC}_i = \big( \text{Nominal}_i + \text{CDC}_{i+1} \big) \cdot {\text{Valid Discount}_{\frac{i}{12}}} 
\Rightarrow \text{CDC}_i = \sum_{\tau=i} \big( \text{Valid Discount}_{\frac{i}{12}} \big)^{\tau-i+1} \cdot \text{Nominal}_\tau 
\end{array}
$$

Thus, the cumulated and discounted cashflow correction for default risk is assigned to the Amount Type Credit Default Risk (CDR) and it is obtained from the Amount Type Claims multiplying the right hand side of the previous formula by the corresponding risk factor

$$
\text{CDC}_i^{\text{CDR}} = \sum_{\tau=i} \big( \text{Valid Discount}_{\frac{i}{12}} \big)^{\tau-i+1} \cdot \text{Nominal}_\tau^{\text{Claim}} \cdot \big( e^{-\gamma(\tau-i)} -1 \big)
$$

where $\gamma$ is the [yearly default rate](../DataModel/DataStructure#credit-default-rate) converted to monthly time step using: 

$$
\gamma = ( 1 + \text{Credit Default Rate}) ^{\frac{1}{12}} ~,
$$

In [0]:
public interface DiscountedCashflow : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? Accidentyear), ImportStorage>
{
    private PeriodType periodType => GetStorage().GetPeriodType(Identity.AmountType, Identity.EstimateType); 
    
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<DiscountedCashflow>(s => s.WithApplicability<DiscountedCreditRiskCashflow>(x => x.Identity.Id.IsReinsurance && x.GetStorage().GetCdr().Contains(x.Identity.AmountType)));

    [NotVisible]
    string EconomicBasis => GetContext();

    [NotVisible]
    double[] MonthlyDiscounting => GetScope<MonthlyRate>(Identity.Id).Discount;
    
    [NotVisible]
    double[] NominalValues => GetScope<NominalCashflow>(Identity).Values;

    double[] Values => Multiply(-1d, NominalValues.ComputeDiscountAndCumulate(MonthlyDiscounting, periodType)); // we need to flip the sign to create a reserve view
}

public interface DiscountedCreditRiskCashflow : DiscountedCashflow
{     
    private string cdrBasis => Identity.AmountType == AmountTypes.CDR ? EconomicBases.C : EconomicBases.L;
    private double nonPerformanceRiskRate => GetStorage().GetNonPerformanceRiskRate(Identity.Id, cdrBasis);
        
    double[] DiscountedCashflow.Values => Multiply(-1d, NominalValues.ComputeDiscountAndCumulateWithCreditDefaultRisk(MonthlyDiscounting, nonPerformanceRiskRate)); // we need to flip the sign to create a reserve view
}

## Telescoping Difference


Present Value figures for a specific period are typically reported through an analysis of change, where for each [AoC Step](#aoc-step-structure) the variation with respect to the preceding AoC Step is shown.

The Telescoping Difference is basically the delta between two adjacent AoC Steps, whereby the [ParentAocStep](#aoc-step-structure) is used to determine the AoC Step. 

It is defined as follows:

$$
\text{TelescopingDifference}_i = 
\text{CDC}_{i}\big(\text{current AoC Step}\big) - \text{CDC}_{i}\big(\text{parent AoC Step}\big)
$$

where AoC Type is the AoC Type of the AoC Step for which the calculations are being performed.

In [0]:
public interface TelescopicDifference : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? Accidentyear), ImportStorage>
{
    [NotVisible]
    string EconomicBasis => GetContext();
    private double[] CurrentValues => GetScope<DiscountedCashflow>(Identity).Values;
    
    private double[] PreviousValues => (GetScope<ParentAocStep>((Identity.Id, Identity.AmountType, InputSource.Cashflow)))
                                            .Values
                                            .Select(aoc => GetScope<DiscountedCashflow>((Identity.Id with {AocType = aoc.AocType, Novelty = aoc.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.Accidentyear)).Values)
                                            .Where(cf => cf.Count() > 0)
                                            .AggregateDoubleArray();
    
    double[] Values => Subtract(CurrentValues, PreviousValues);
}

## Valuation Period 

The present value ($\rm{PV}$) can be determined by taking the appropriate elements of the cumulated discounted cash flows. This is done as function of the two [projection parameters](../DataModel/DataStructure#projection-configuration) $\rm{Shift}$ ($S$) and $\rm{TimeStep}$ ($TS$):

$$
\text{PV}(S, TS) = \left\{
\begin{array}{cl}
\text{PV}_{S}  ~, & \text{if Valuation Period is Beginning of Period} \\
\text{PV}_{S+TS/2 -1}  ~, & \text{if Valuation Period is Mid of Period} \\
\sum_{i=S}^{S + TS - 1}\text{PV }_{i}  ~, & \text{if Valuation Period is Delta} \\
\text{PV}_{S + TS}  ~, & \text{if Valuation Period is End of Period} \\
\end{array}
\right.
$$

where the term $TS/2$ uses MidpointRounding.AwayFromZero (as defined by *https:[]()//docs.microsoft.com/en-us/dotnet/api/system.midpointrounding?view=net-6.0)*: rounding to the nearest number, away from zero in the exact halfway case. Furthermore, if the array is smaller than the index, then the last element is returned.

For instance, for the current year and year-to-date view we have $S=0$ and $TS=3$ for the first quarter, $TS=6$ for the 2nd quarter and so on.
For the projection values of next year first quarter we would have $S=12$ and $TS=3$, etc.

In [0]:
public interface IWithGetValueFromValues : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    private int shift => GetStorage().GetShift(Identity.Id.ProjectionPeriod);
    private int timeStep => 
        Identity.Id.LiabilityType == LiabilityTypes.LRC && 
        Identity.AccidentYear.HasValue && 
        (MonthInAYear * Identity.AccidentYear == (MonthInAYear * GetStorage().CurrentReportingPeriod.Year + GetStorage().GetShift(Identity.Id.ProjectionPeriod)))
    ? int.MaxValue
    : GetStorage().GetTimeStep(Identity.Id.ProjectionPeriod);

    public double GetValueFromValues(double[] Values, string overrideValuationPeriod = null)
    {
        var valuationPeriod = Enum.TryParse(overrideValuationPeriod, out ValuationPeriod ret) ? ret : GetStorage().GetValuationPeriod(Identity.Id);
        return valuationPeriod switch {
                        ValuationPeriod.BeginningOfPeriod => Values.ElementAtOrDefault(shift),
                        ValuationPeriod.MidOfPeriod => Values.ElementAtOrDefault(shift + Convert.ToInt32(Math.Round(timeStep / 2d, MidpointRounding.AwayFromZero)) - 1),
                        ValuationPeriod.Delta => Values.Skip(shift).Take(timeStep).Sum(),
                        ValuationPeriod.EndOfPeriod  => Values.ElementAtOrDefault(shift + timeStep),
                        ValuationPeriod.NotApplicable => default
                    };
    }
}

## Interest Accretion

Since the Interest Accretion cash flows are typically not provided as input (as they can be computed from its parent AoC Step), its present values can be computed as follows:

$$
\text{InterestAccretion}_i(\text{AoC step}) = \left\{
\begin{array}{cl}
\big(\text{CDC}_i(\text{Parent AoC step}) - \text{Nominal}_i(\text{parent AoC step}) \big) \cdot \big({\text{Valid Interest}_{\frac{i}{12}}} - 1 \big)~, & \text{if Amount Type's Transaction Period is Beginning of Period} \\
\text{CDC}_i(\text{parent AoC step}) \cdot \big({\text{Valid Interest}_{\frac{i}{12}}} - 1 \big)~, & \text{otherwise}
\end{array}
\right.
$$

<br> Due to the Credit Default Risk of a reinsurance partner, the logic to compute the Interest Accretion for this Amount Type must be defined separately. Since it is based on Claims, the Period Type is implicitly defined.

In [0]:
public interface IWithInterestAccretion : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    private double[] parentDiscountedValues => Multiply(-1d, GetScope<DiscountedCashflow>(Identity).Values);    
    private double[] parentNominalValues => GetScope<NominalCashflow>(Identity).Values;
    private double[] monthlyInterestFactor => GetScope<MonthlyRate>(Identity.Id).Interest;
    
    double[] GetInterestAccretion() 
    {
        if(!monthlyInterestFactor.Any())
            return Enumerable.Empty<double>().ToArray();
        var periodType = GetStorage().GetPeriodType(Identity.AmountType, Identity.EstimateType);
        var ret = new double[parentDiscountedValues.Length];
        
        switch (periodType) {
            case PeriodType.BeginningOfPeriod :
                for (var i = 0; i < parentDiscountedValues.Length; i++)
                     ret[i] = -1d * (parentDiscountedValues[i] - parentNominalValues[i]) * (monthlyInterestFactor.GetValidElement(i/12) - 1d );
                break;
            default :
                for (var i = 0; i < parentDiscountedValues.Length; i++)
                     ret[i] = -1d * parentDiscountedValues[i] * (monthlyInterestFactor.GetValidElement(i/12) - 1d );
             break;
        }
        
        return ret;
    }
}

public interface IWithInterestAccretionForCreditRisk : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    private double[] nominalClaimsCashflow => GetScope<AllClaimsCashflow>(Identity).Values;
    private double[] nominalValuesCreditRisk => Multiply(-1, GetScope<CreditDefaultRiskNominalCashflow>(Identity with {Id = Identity.Id with {AocType = AocTypes.CF}}).Values);
    private double[] monthlyInterestFactor => GetScope<MonthlyRate>(Identity.Id).Interest;
    private string cdrBasis => Identity.AmountType == AmountTypes.CDR ? EconomicBases.C : EconomicBases.L;
    private double nonPerformanceRiskRate => GetStorage().GetNonPerformanceRiskRate(Identity.Id, cdrBasis);
    
    double[] GetInterestAccretion() 
    {
        if(!monthlyInterestFactor.Any())
            return Enumerable.Empty<double>().ToArray();
            
        var interestOnClaimsCashflow = new double[nominalClaimsCashflow.Length];
        var interestOnClaimsCashflowCreditRisk = new double[nominalClaimsCashflow.Length];
        var effectCreditRisk = new double[nominalClaimsCashflow.Length];
        for (var i = nominalClaimsCashflow.Length - 1; i >= 0; i--) {
            interestOnClaimsCashflow[i] = 1 / monthlyInterestFactor.GetValidElement(i/12) * (interestOnClaimsCashflow.ElementAtOrDefault(i + 1) + nominalClaimsCashflow[i] - nominalClaimsCashflow.ElementAtOrDefault(i + 1));
            interestOnClaimsCashflowCreditRisk[i] = 1 / monthlyInterestFactor.GetValidElement(i/12) * (Math.Exp(-nonPerformanceRiskRate) * interestOnClaimsCashflowCreditRisk.ElementAtOrDefault(i + 1) + nominalClaimsCashflow[i] - nominalClaimsCashflow.ElementAtOrDefault(i + 1));
            effectCreditRisk[i] = interestOnClaimsCashflow[i] - interestOnClaimsCashflowCreditRisk[i];
        }
            
        return Subtract(nominalValuesCreditRisk, effectCreditRisk);
    }
}

In case the Interest Accretion step is computed without using a cash flow like for In Force contribution in Contractual Service Margin and Loss Component computed by the [Technical Margin](4ImportScope-TechnicalMargin#technical-margin). Interest Accretion factor is computed by: 

$$
IAF = \prod _{i = S}^{S + TS - 1} \text{Monthly }IF_i ~,
$$
where $\text{Monthly }IF_i$ is computed through the [Yield Curve](#interest-and-discount-rates-and-factors) using a specified Economic Basis (Current or Locked-in) specified in the scope computation that uses this factor. The current Interest Accretion factor is applied to the aggregated value over the previus AoC Steps to compute the Interest Accreation amount. 

In [0]:
public interface InterestAccretionFactor : IScope<ImportIdentity, ImportStorage>{
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod);
    private int shift => GetStorage().GetShift(Identity.ProjectionPeriod);
    
    double GetInterestAccretionFactor(string economicBasis) 
    {
        double[] monthlyInterestFactor = GetScope<MonthlyRate>(Identity, o => o.WithContext(economicBasis)).Interest;
        return Enumerable.Range(shift,timeStep).Select(i => monthlyInterestFactor.GetValidElement(i/12)).Aggregate(1d, (x, y) => x * y ) - 1d;
    }
}

### New Business Interest Accretion for Liabilities

For in-force liabilities positions (i.e. Novelty is I), interest can be accreted from the beginning of the period, as the in-force reserve is already present from day one.
Instead, new business (and perhaps other Novelties behaving similarly) comes during the period, and therefore the interest accretion might use a different logic.
The most commong approach is to use the nominal cash flows and recalculate the interest accordingly. The Scope below can be used as reference, but it is not active in the default calculation engine.

In [0]:
public interface NewBusinessInterestAccretion : IScope<ImportIdentity, ImportStorage>
{
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod);
    private int shift => GetStorage().GetShift(Identity.ProjectionPeriod);

    double GetInterestAccretion(double[] values, string economicBasis) 
    {
        var monthlyInterestFactor = GetScope<MonthlyRate>(Identity, o => o.WithContext(economicBasis)).Interest;
        return values.NewBusinessInterestAccretion(monthlyInterestFactor, timeStep, shift);
    }
}

## Present Value

The PV values are valid for all choices of the [Economic Basis](../DataModel/DataStructure#economic-basis):

$$
\text{PV}_i (\text{AoC step}) = \left\{
\begin{array}{rl}
\text{CDC}_i ~, & \text{if AoC Type = BOP} \\
-\text{Nominal}_i(\text{Parent AoC Step}) ~, & \text{if AoC Type = CF } \\
\text{InterestAccretion}_i ~, & \text{if AoC Type = IA } \\
0 ~, & \text{if AoC Type = AM } \\\
\text{CDC}_i(\text{Parent AoC step}) ~, & \text{if AoC Type = EOP } \\
\text{TelescopingDifference}_i ~, & \text{otherwise}
\end{array}
\right.
$$

where $i$ denotes the months, and the [$\rm{TelescopingDifference}_i$](#telescopic-difference) and the [$\rm{InterestAccretion}_i$](#interest-accretion) quantities are defined above.

In [0]:
public interface PresentValue : IWithGetValueFromValues {   
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PresentValue>(s => s
            .WithApplicability<ComputePresentValueWithIfrsVariable>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow || x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode))
            .WithApplicability<PresentValueFromDiscountedCashflow>(x => (x.Identity.Id.AocType == AocTypes.BOP && x.Identity.Id.Novelty != Novelties.C) || x.Identity.Id.AocType == AocTypes.EOP)
            .WithApplicability<CashflowAocStep>(x => x.Identity.Id.AocType == AocTypes.CF)
            .WithApplicability<PresentValueWithInterestAccretion>(x => x.Identity.Id.AocType == AocTypes.IA)
            .WithApplicability<EmptyValuesAocStep>(x => ComputationHelper.AocStepWithNoPv.Contains(new AocStep(x.Identity.Id.AocType, x.Identity.Id.Novelty)) ||
                                                        (x.Identity.Id.AocType == AocTypes.CRU && !x.GetStorage().GetCdr().Contains(x.Identity.AmountType)) )
            );
    
    [NotVisible][IdentityProperty][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => GetContext();
    
    [NotVisible]
    double[] Values => GetScope<TelescopicDifference>(Identity).Values;
    
    public double Value => GetValueFromValues(Values);
}

public interface ComputePresentValueWithIfrsVariable : PresentValue {
    double PresentValue.Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, EconomicBasis, Identity.AccidentYear, Identity.Id.ProjectionPeriod);
    double[] PresentValue.Values => Enumerable.Empty<double>().ToArray();
}

public interface PresentValueFromDiscountedCashflow : PresentValue {
    [NotVisible] double[] PresentValue.Values => GetScope<DiscountedCashflow>(Identity).Values;
}

public interface CashflowAocStep : PresentValue {
    [NotVisible] double[] PresentValue.Values => GetScope<NominalCashflow>(Identity).Values;
}

public interface PresentValueWithInterestAccretion : PresentValue, IWithInterestAccretion {
     static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PresentValueWithInterestAccretion>(s => s.WithApplicability<PresentValueWithInterestAccretionForCreditRisk>(x => x.Identity.Id.IsReinsurance && x.GetStorage().GetCdr().Contains(x.Identity.AmountType)));    
    [NotVisible] double[] PresentValue.Values => GetInterestAccretion();
}

public interface PresentValueWithInterestAccretionForCreditRisk : PresentValue, IWithInterestAccretionForCreditRisk {
    [NotVisible] double[] PresentValue.Values => GetInterestAccretion();
}

public interface EmptyValuesAocStep : PresentValue {
    [NotVisible] double[] PresentValue.Values => Enumerable.Empty<double>().ToArray();
}

## Wrapper over Accident Year
Present Value are here collected for all available Accident Year for any combination of [Amount Type](../DataModel/DataStructure#amount-type) and [Estimate Type](../DataModel/DataStructure#estimate-type) specified in input (scope identity). In addition, the sum of present values over the available Accident Years is also provided. 

In [0]:
public interface PvAggregatedOverAccidentYear : IScope<(ImportIdentity Id, string AmountType, string EstimateType), ImportStorage>
{   
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => GetContext();
        
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.Id.DataNode, Identity.Id.ProjectionPeriod).ToArray();  
    
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => accidentYears.Select(ay => 
        (Identity.AmountType, Identity.EstimateType, ay, GetScope<PresentValue>((Identity.Id, Identity.AmountType, Identity.EstimateType, ay), o => o.WithContext(EconomicBasis)).Value))
        .ToArray();
            
    double Value => PresentValues.Sum(pv => pv.Value);
}

# Best Estimate

PV Current and PV Locked below basically hold the Present Values [PV](#present-value) computed using the **Current** yield curves and the **Locked** yield curves, respectively.

Values are available for each AmountType and AccidentYear (by calling <code>PvLocked.PresentValues</code>):

$$
\text{PV Locked}(\text{AoC step}, \text{Amount Type}) = \text{PV}(\text{AoC step}, \text{Amount Type})|_{\text{Economic Base = L}}
$$

$$
\text{PV Current}(\text{AoC step}, \text{Amount Type}) = \text{PV}(\text{AoC step}, \text{Amount Type})|_{\text{Economic Base = C}}
$$

Aggregated present values are also available through summing over Amount Types and Accident Year (by calling <code>PvLocked.PresentValues.Sum(pv => pv.Value)</code>).

PV Current and PV Locked are both computed in the BBA methodology, although the CSM calculations uses only PV Locked for BBA valuation approach and PV Current for VFA valuation approach. Depending on the valuation approach Present Values Locked and/or Present Values current are stored in the database under the [IfrsVariable](../DataModel/DataStructure#ifrs-variable) data structure.

In [0]:
public interface PvLocked : IScope<ImportIdentity, ImportStorage>
{   
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.L;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.BE;
       
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
        .SelectMany(at => GetScope<PvAggregatedOverAccidentYear>((Identity, at, EstimateType), o => o.WithContext(EconomicBasis)).PresentValues
        ).ToArray();
}

In [0]:
public interface PvCurrent : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.C;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.BE;

    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
        .SelectMany(at => GetScope<PvAggregatedOverAccidentYear>((Identity, at, EstimateType), o => o.WithContext(EconomicBasis)).PresentValues
        ).ToArray();
}

In [0]:
public interface CumulatedNominalBE : IScope<ImportIdentity, ImportStorage> {  
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.N;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.BE;
    
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
        .SelectMany(at => GetScope<PvAggregatedOverAccidentYear>((Identity, at, EstimateType), o => o.WithContext(EconomicBasis)).PresentValues
        ).ToArray();
}

# Risk Adjustment

Risk Adjustment values ($\rm{RA}$) are accessible from the [PresentValue](#present-value) data and have [Estimate Type](../DataModel/DataStructure#estimate-type) $RA$. In particular, the Locked-In and Current values are given by:

$$
\text{RA Locked}(\text{AoC step}) = \text{PV}(\text{AoC step})|_{\text{Calculation Type = RA},~ \text{Economic Basis = L}}
$$

$$
\text{RA Current}(\text{AoC step}) = \text{PV}(\text{AoC step})|_{\text{Calculation Type = RA},~ \text{Economic Basis = C}}
$$

where PV is defined [above](#present-value) and uses the input cash flows with Calculation Type = RA.

In [0]:
public interface RaLocked : IScope<ImportIdentity, ImportStorage>
{   
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.L;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.RA;

    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => GetScope<PvAggregatedOverAccidentYear>((Identity, (string)null, EstimateType), o => o.WithContext(EconomicBasis)).PresentValues;
}

In [0]:
public interface RaCurrent : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.C;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.RA;
    
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => GetScope<PvAggregatedOverAccidentYear>((Identity, (string)null, EstimateType), o => o.WithContext(EconomicBasis)).PresentValues;
}

In [0]:
public interface CumulatedNominalRA : IScope<ImportIdentity, ImportStorage> {  
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.N;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.RA;
    
    [NotVisible]
    (string AmountType, string EstimateType, int? AccidentYear, double Value)[] PresentValues => GetScope<PvAggregatedOverAccidentYear>((Identity, (string)null, EstimateType), o => o.WithContext(EconomicBasis)).PresentValues;
}

# Amortization

For the Amortization AoC Step (AoC Type **AM**), the amortization factors to be used are defined below.

## Monthly AM Factor

The calculation of the Amortization factors uses a release pattern defined in input. 
A generic pattern specific for a certain GIC can be input through the [Single Data Node Parameter](../DataModel/#data-node-parameters) through the Release Pattern fields. It is possible to also define release pattern specific for an AmountType. For example a coverage units pattern for the calculaiton of Contractual service margin, a premium pattern for the Premium allocation approach, and a deferral pattern for deferrable expenses. 
The calculation gives priority to the specific pattern and falls back to the general pattern provided as Data node parameter if absent. 

**Specific AmountType**

For valuation approaches BBA and VFA the coverage unit (CU) of a GIC is introduced in the standard as the quantity of the service provided in that GIC. The service is measured by considering the quantity of benefits provided as well as the expected coverage period of the GIC.

The cash flows of coverage units are retrieved from the discounted (with Locked-in, Current, or undiscounted) and cumulated cash flows with [EstimateType](../DataModel/DataStructure#estimate-type) P and [AmountType](../DataModel/DataStructure#amount-type) CU. The pattern for the AoC Step CL,C is considered.  

**Calculation**

For a certain GIC, the monthly Amortization Factors $\text{Monthly }AF_i$ are computed from the provided pattern for that GIC:

$$
\text{Monthly }AF_i = Max\left(0, 1 - \frac{ \text{Nominal}_i} {\text{CDC}_i }\right) ~.
$$

where:
- $i$ denotes a monthly period;
- the nominal pattern $\text{Nominal}_i$ is the monthly nominal pattern provided as input data;
- $\text{CDC}_i$ is the corresponding cumulated discounted cash flows as defined [above](#cumulated-discounted-cash-flows). The discounting factors (Locked-in, Current, Nominal or undiscounted) is controlled by the EconomicBasis parameter following Ifrs 17 standards : Locked-in for BBA, Current for VFA, Current for PAA-LIC or taken from [EconomicBasisDriver](../DataModel/DataStructure#data-node-parameters) when the choice is left to the user. 

Occasionally, it is required to shift the relase pattern to a certain period in order to correctly compute the release. Here we allow the $\text{Nominal}_i$ to be shifted arbitrarily. The shift is considered also in the calculationof the corresponding $\text{CDC}_i$ term and it is controlled by the computation of the specific release (Deferral, Premium, Contractual service margin).

In [0]:
public interface MonthlyAmortizationFactorCashflow : IScope<(ImportIdentity Id, string AmountType, int patternShift), ImportStorage>
{
    (string EffectiveAmountType, double[] Values) releasePattern => GetStorage().GetReleasePattern(Identity.Id, Identity.AmountType, Identity.patternShift);

    private PeriodType periodType => GetStorage().GetPeriodType(Identity.AmountType, EstimateTypes.P);
    private double[] monthlyDiscounting => GetScope<MonthlyRate>(Identity.Id).Discount;
    private double[] cdcPattern => releasePattern.Values.ComputeDiscountAndCumulate(monthlyDiscounting, periodType); 
    
    [NotVisible] string EconomicBasis => GetContext();
    
    double[] MonthlyAmortizationFactors => Identity.Id.AocType switch {
        AocTypes.AM when releasePattern.Values?.Any() ?? false => releasePattern.Values.Zip(cdcPattern,  //Extract to an other scope with month in the identity to avoid Zip?
            (nominal, discountedCumulated) => Math.Abs(discountedCumulated) >= Precision ? Math.Max(0, 1 - nominal / discountedCumulated) : 0).ToArray(),
        _ => Enumerable.Empty<double>().ToArray(),
        };
}

## Current Period AM actor
For a certain projection period - defined by the Shift, $S$, and the Time-Step, $TS$, parameters - the Amortization Factor is then given by the product of the corresponding monthly amortization factors:

$$
AF = 1 - \prod _{i = S}^{S + TS - 1} \text{Monthly }AF_i ~.
$$

Each GIC will have his own AF.

In order to run off the business of a given Group of Contract one should provide a cash flow of 0s for the AoC Step with AoC Type CL and Novelty C. When computing the AF this results in the product of the monthly amortization factors of the period to be 1. In this case, the computed AF does not follow the formula above but is 1 allowing for the full release of the Technical Margin in the AM AoC Step.

In [0]:
public interface CurrentPeriodAmortizationFactor : IScope<(ImportIdentity Id, string AmountType, int patternShift), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<CurrentPeriodAmortizationFactor>(s => 
                 s.WithApplicability<AmfFromIfrsVariable>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow
                                                            || x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)));

    private int shift => GetStorage().GetShift(Identity.Id.ProjectionPeriod);
    private int timeStep => GetStorage().GetTimeStep(Identity.Id.ProjectionPeriod);
    private double amortizedFactor => GetScope<MonthlyAmortizationFactorCashflow>(Identity)
                            .MonthlyAmortizationFactors
                            .Skip(shift)
                            .Take(timeStep)
                            .DefaultIfEmpty()
                            .Aggregate(1d, (x, y) => x * y);
                            
    [NotVisible] string EconomicBasis => GetContext();

    string EstimateType => EstimateTypes.F;
    string EffectiveAmountType => GetScope<MonthlyAmortizationFactorCashflow>(Identity).releasePattern.EffectiveAmountType;
    double Value => 1d - amortizedFactor;
}

public interface AmfFromIfrsVariable : CurrentPeriodAmortizationFactor{
    private double amortizationFactorForAmountType => GetStorage().GetValue(Identity.Id, Identity.AmountType, EstimateType, EconomicBasis, 
        Identity.patternShift == 0 ? null : Identity.patternShift, Identity.Id.ProjectionPeriod); //TODO shift of 0 is a valid value
    
    private double amortizationFactorFromPattern => GetStorage().GetValue(Identity.Id, null, EstimateType, EconomicBasis, Identity.patternShift == 0 ? null : Identity.patternShift, Identity.Id.ProjectionPeriod);
    
    private double amortizationFactorForCu => GetStorage().GetValue(Identity.Id, AmountTypes.CU, EstimateType, EconomicBasis, 
        Identity.patternShift == 0 ? null : Identity.patternShift, Identity.Id.ProjectionPeriod);

    double CurrentPeriodAmortizationFactor.Value => Math.Abs(amortizationFactorForAmountType) >= Precision ? amortizationFactorForAmountType 
        : Math.Abs(amortizationFactorFromPattern) >= Precision ? amortizationFactorFromPattern : amortizationFactorForCu;
    string CurrentPeriodAmortizationFactor.EffectiveAmountType => Math.Abs(amortizationFactorForAmountType) >= Precision ? Identity.AmountType 
        : Math.Abs(amortizationFactorFromPattern) >= Precision ? null : AmountTypes.CU;
}